***

**<center><font size = "6">Manage Data Models in Looker<center>**
***
<center><font size = "2">Prepared by: Sitsawek Sukorn<center>

### Modularizing LookML Code with Extends

### Extend a view to add columns from another view

#### Create a new view

- Click the toggle button to enter Development mode.

- On the Develop tab, select the qwiklabs-ecommerce LookML project.

- Click (+) next to File Browser, and select Create View.

- Name the view location, drag it under the views folder, and add the following code to it:

In [ ]:
view: location {
  extension: required
  dimension: city {
    type: string
    sql: ${TABLE}.city ;;
  }
  dimension: state {
    type: string
    sql: ${TABLE}.state ;;
    map_layer_name: us_states
  }
  dimension: zip {
    type: zipcode
    sql: ${TABLE}.zip ;;
  }
  dimension: country {
    type: string
    map_layer_name: countries
    sql: ${TABLE}.country ;;
  }
  
  dimension: latitude {
    type: number
    sql: ${TABLE}.latitude ;;
  }
  
  dimension: longitude {
    type: number
    sql: ${TABLE}.longitude ;;
  }
}

Notice line 2 (extension: required), which means that this view cannot be joined to other views, and thus will not be visible to users. To use this view, you must integrate it into another view using the extends parameter, which you do in the next section.

Also notice that, unlike with other views, you do not need to include the parameter sql_table_name in the view definition to identify which table to use for the data. Instead, this view will use the table specified in the view that will be extended in the next section.



- Click Save Changes, and then click Validate LookML. No LookML errors were found, and your file should resemble the following:

#### Add extends

- Open the users.view file.

- On a new line at the top of the file (line 1), add the following code, which indicates that the users view is being extended using the location view:

In [ ]:
include: location.view

- On line 3 above sql_table_name, add the following code:

In [ ]:
extends: [location]

Note: Because the extends are added, the sql_table_name parameter identifies which table to use as the data source for both the existing objects in the file and the objects that are added from geography via the extend.

- Remove the existing dimension definitions for: city, country, latitude, longitude, state, and zip (this is the existing order in the file). Instead of being explicitly defined in the users.view file, these dimensions are integrated via the extend from location.view.

- Click Save Changes, and then click Validate LookML.

- Open the event.view file.

- On a new line at the top of the file (line 1), add the following code:

In [ ]:
include: location.view

- On line 3 above sql_table_name, add the following code:

In [ ]:
extends: [location]

- As you did with the users view, remove the existing dimension definitions for: city, country, latitude, longitude, state, and zip.

! must move location.view to view if not that errors. in this lab not mention that.

#### Test the extended view for Users and Events in the Order Items Explore

- Navigate to the Explore page for Order Items.

- From the Users view, select the City, Country, Latitude, Longitude, State, and Zip dimensions.

- Click Run.

Even though you removed the definitions for these dimensions (city, country, latitude, longitude, state and zip) from the users.view file, you can see and use them because they were added to the users.view file using an extend from the location.view file!

- Navigate to the Events Explore.

- From the Events view, select the City, Country, Latitude, Longitude, State, and Zip dimensions.

Again, even though you removed the definitions for these dimensions from the events.view file, you can see and use them because they were added to the events.view file using an extend.

- Navigate back to the events.view file in the Looker IDE.

#### Commit changes and deploy to production


- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

### Extend an Explore to add joins from another Explore


- Navigate to the training_ecommerce.model file.

- After the order_items Explore definition (around line 43), create a new base Explore called base_events, using the following code:

In [ ]:
explore: base_events {
  extension: required
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }
}

Notice again the line for extension: required, which means that this Explore is not visible to users. Your file should resemble the following:

Next, you'll modify the existing definition for the events Explore to extend it with the views from base_events.

- From the events Explore, remove the existing joins for event_session_facts and users.

These joined views are integrated from the base_events Explore via code added in the next step. Notice that the join definition for event_session_funnel remains to customize this Explore for a particular set of users. Your file should resemble the following:

In [ ]:
explore: events {
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }

  to

  explore: events {
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }

- Under the first line of the events Explore definition, add the following code:

In [ ]:
description: "Start here for Event analysis"
  fields: [ALL_FIELDS*]
from: events
  view_name: events
  extends: [base_events]

#to

explore: events {
  description: "Start here for Event analysis"
  fields: [ALL_FIELDS*]
  from: events
  view_name: events
  extends: [base_events]
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }

Note: The new lines provide a description for the Explore info button, identify which fields from which view file to include (all fields), and specify which Explore is being used to extend the events Explore.

Your final definition for the events Explore should resemble the following:

In [ ]:
explore: events {
  description: "Start here for Event analysis"
  fields: [ALL_FIELDS*]
  from: events
  view_name: events
  extends: [base_events]
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
}

Note: The from and view_name are both pointing to the events view, so why include both? The from makes sure that you are using the original view called events (not an alias name for the view and not an extended one), and the view_name is the view file name, which could be an alias, etc.

- Below the modified events Explore definition, to add a new Explore called conversions, use the following code:

In [ ]:
explore: conversions {
  description: "Start here for Conversion Analysis"
  fields: [ALL_FIELDS*, -order_items.total_revenue_from_completed_orders]
  from: events
  view_name: events
  extends: [base_events]
  join: order_items {
    type: left_outer
    sql_on: ${users.id} = ${order_items.user_id} ;;
    relationship: many_to_many
  }
}

Lines 2-6 here provide a description for the Explore info button, identify which fields from which view file to include (all fields except the total_revenue_from_completed_orders measure in the order items view), and specify which Explore is being used to extend this Explore (i.e., the same base_events Explore that was used to extend the events Explore).

- Click Save Changes, and then click Validate LookML. No LookML errors were found, and your file should resemble the following:

Now it's time to test your new Explores. Go to each Explore (Events and Conversions), and notice which views are included. Because the Explores share a core set of views but are customized with additional views, each one serves a different user audience.

- Navigate to the Events Explore, which contains the views joined in the base Explore (Events, Event Session Facts, Users) plus the Event Session Funnel view.

- To review the description, hold the pointer over Information (info button) next to Events.

- Navigate to the Conversions Explore, which contains the views joined in the base Explore (Events, Event Session Facts, Users) plus the Order Items view.

- To review the description, hold the pointer over Information (info button) next to Conversions.

- Review the measures in the Order Items view; total_revenue_from_completed_orders is not listed.

- Return to the training_ecommerce.model file in the Looker IDE.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

***

**<center><font size = "6">Troubleshooting Data Models in Looker<center>**
***

### Use SQL Runner to explore available data and troubleshoot SQL queries

#### Connect to a BigQuery dataset in SQL Runner

- Click the toggle button to enter Development mode.

- On the Develop tab, select SQL Runner.

- Click Settings (settings-gear-icon.png), and then click Search public projects. The box for Project is now empty.

- Type cloud-training-demos, and press ENTER.

- For Dataset, select looker_ecomm. A list of the available tables in this BigQuery dataset is displayed.

- Add the following query to the SQL Query window:

In [ ]:
SELECT
  orders.user_id as user_id
 	,COUNT(*) as lifetime_orders
 	,SUM(orders.order_price) as lifetime_sales
FROM cloud-training-demos.looker_ecomm.orders
GROUP BY user_id
LIMIT 10

Note: The provided SQL query contains incorrect information, which you troubleshoot in the next steps.

- Click Run. The following error message is displayed:

Query execution failed: - Not found: Table cloud-training-demos:looker_ecomm.orders was not found in location US

#### Identify the correct table names for SQL queries

- In the list of table names under Tables, select the table that contains order information.

- Update the table name in the SQL query:

In [ ]:
SELECT
  order_items.user_id as user_id
 	,COUNT(*) as lifetime_orders
 	,SUM(order_items.order_price) as lifetime_sales
FROM cloud-training-demos.looker_ecomm.order_items
GROUP BY user_id
LIMIT 10

- Click Run. A new error message is displayed:

Query execution failed: - Name order_price not found inside order_items at [5:19]

- Under Tables, click order_items. A list of the table columns is displayed.

- Update the column name in the SQL query:

In [ ]:
SELECT
  order_items.user_id as user_id
 	,COUNT(*) as lifetime_orders
 	,SUM(order_items.sale_price) as lifetime_sales
FROM cloud-training-demos.looker_ecomm.order_items
GROUP BY user_id
LIMIT 10

- Click Run. The query results are returned successfully. You can now save this query as a SQL derived table.

#### Save the query as a SQL derived table

- Click Settings (settings-gear-icon.png) next to Run, and then click Add to Project.

- For Project, select qwiklabs-ecommerce.

- For View Name, type user_order_lifetime.

- Click Add.

- In the File Browser, drag user_order_lifetime.view to to the views folder.

- In the user_order_lifetime.view file, delete the code line for LIMIT 10 from the sql parameter.

Note: You are not defining a primary_key for the view at this time.

In [ ]:
view: user_order_lifetime {
  derived_table: {
    sql: SELECT
        order_items.user_id as user_id
         ,COUNT(*) as lifetime_orders
         ,SUM(order_items.sale_price) as lifetime_sales
      FROM cloud-training-demos.looker_ecomm.order_items
      GROUP BY user_id
      LIMIT 10 # <-- delete here
       ;;
  }

  measure: count {
    type: count
    drill_fields: [detail*]
  }

  dimension: user_id {
    type: number
    sql: ${TABLE}.user_id ;;
  }

  dimension: lifetime_orders {
    type: number
    sql: ${TABLE}.lifetime_orders ;;
  }

  dimension: lifetime_sales {
    type: number
    sql: ${TABLE}.lifetime_sales ;;
  }

  set: detail {
    fields: [user_id, lifetime_orders, lifetime_sales]
  }
}


- Click Save Changes, and then click Validate LookML. No LookML errors were found, and your file should resemble the following:

- Navigate to the training_ecommerce.model file.

- In the explore: events definition, on a new line before join: event_session_facts, use the following code to define a new join:

In [ ]:
join: user_order_lifetime {
  type: left_outer
  sql_on: ${events.user_id} = ${user_order_lifetime.user_id};;
  relationship: many_to_one
}

In [ ]:
# to this 

connection: "bigquery_public_data_looker"

# include all the views
include: "/views/*.view"
include: "/z_tests/*.lkml"
include: "/**/*.dashboard"

datagroup: training_ecommerce_default_datagroup {
  # sql_trigger: SELECT MAX(id) FROM etl_log;;
  max_cache_age: "1 hour"
}

persist_with: training_ecommerce_default_datagroup

label: "E-Commerce Training"

explore: order_items {
  join: users {
    type: left_outer
    sql_on: ${order_items.user_id} = ${users.id} ;;
    relationship: many_to_one
  }

  join: inventory_items {
    type: left_outer
    sql_on: ${order_items.inventory_item_id} = ${inventory_items.id} ;;
    relationship: many_to_one
  }

  join: products {
    type: left_outer
    sql_on: ${inventory_items.product_id} = ${products.id} ;;
    relationship: many_to_one
  }

  join: distribution_centers {
    type: left_outer
    sql_on: ${products.distribution_center_id} = ${distribution_centers.id} ;;
    relationship: many_to_one
  }
}

explore: events {
  join: user_order_lifetime {
    type: left_outer
    sql_on: ${events.user_id} = ${user_order_lifetime.user_id};;
    relationship: many_to_one
  }
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }
}


#### Commit changes and deploy to production


- Click Validate LookML and then click Commit Changes and Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

### Use the LookML Validator to test syntax and validate relationships defined in the model

#### Create a new dimension by referencing dimensions in another view

- In the qwiklabs-ecommerce project, open users.view.

- Find the last dimension, and add the following code (around line 88) to create a new dimension:

In [ ]:
dimension: average_sales  {
  type: ${number}
  sql: user_order_lifetime.lifetime_sales /
    user_order_lifetime.lifetime_orders ;;
  value_format_name: usd
}

Notice that the new dimension references dimensions from the newly created view called user_order_lifetime.

Note: This LookML code contains incorrect syntax, which you troubleshoot in the next steps.

- Click Save Changes, and then click Validate LookML. Four different errors are displayed:

- The first item identifies the location of the syntax error as “Invalid LookML syntax near line 89”.

- The last item identifies the specific syntax error: “Expecting ‘keyword’, ‘}’, got ‘identifier’

Because the view now contains invalid syntax, it is no longer considered a valid view; thus, additional errors are identified within the model file because the view cannot be found:

- “Join name must match a view name” and “Could not find a field named users.id”


After the syntax for the new dimension in the view file is corrected, these model file errors are also resolved because the view is again valid.

#### Identify and correct syntax errors in new dimensions

You now troubleshoot the errors in the LookML code.

- Review the other dimensions in the users.view file.

- Review the documentation on defining numeric dimensions.
https://cloud.google.com/looker/docs/reference/param-dimension-filter-parameter-types#number

- Update the LookML code for the dimension:

In [ ]:
dimension: average_sales  {
   type: number
   sql: ${user_order_lifetime.lifetime_sales} /
     ${user_order_lifetime.lifetime_orders} ;;
   value_format_name: usd
  }

- Click Save Changes, and then click Validate LookML.

Because the syntax was updated, the view is valid again, and the model errors have also been resolved. However, there are now two new errors:

- Review the second error message for Inaccessible view in the Looker error catalog. There are a few possible options to investigate:

- The view doesn’t exist.
- The view is not joined correctly to the explore.

- Review the list of views in the File Browser. Notice that the view called user_order_lifetime actually exists in the File Browser.

- Open and review training_ecommerce.model.

Notice that the new view called user_order_lifetime is not joined to the base views of the Explores in the model file. Also notice that the users.view is joined to both the order_items and event Explores. For this reason, **the new view for user_order_lifetime must also be joined to both Explores in order for the new dimension to be defined successfully within users.view**.

- In the explore: order_items definition, on a new line before join: users, use the following code to define a new join:

In [ ]:
join: user_order_lifetime {
  type: left_outer
  sql: ${order_items.user_id} = ${user_order_lifetime.user_id};;
  relationship: many_to_one
}

Note: One of these joins is incorrectly defined, which you troubleshoot in the next section.

- Click Save Changes, and then click Validate LookML.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes and Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

### Use the Explore query window to diagnose missing objects and error messages

#### Review a new view within the Explore

- Open a new Looker window in a new tab.

- Navigate to Explore > Order Items.

- Expand the view for User Order Lifetime.

Three dimensions are displayed, lifetime_orders, lifetime_sales, user_id, but no measures.

Leave this browser tab open as you continue to the next steps.

#### Identify and correct missing parameters for aggregations within a view


- Return to the browser tab for the Looker IDE, and navigate back to user_order_lifetime.view. One measure called count should be displayed in the Explore.

- Review the documentation on requirements for symmetric aggregates. The first requirement is that all views involved in the join need to have a primary_key defined.

- Review user_order_lifetime.view again. There is no primary_key defined.

- Within user_order_lifetime.view, update the user_id dimension to define it as the primary_key for the view:

In [ ]:
dimension: user_id {
  primary_key: yes
  type: number
  sql: ${TABLE}.user_id ;;
}

- Click Save Changes, and then click Validate LookML.

- Return to the browser tab for the Order Items Explore, and refresh the page.

- Expand the view for User Order Lifetime. After you define a primary_key for the view, the Count measure is now displayed.

In the next steps, you remain in the Order Items Explore to test the new dimension defined in users.view (Average Sales) that relies on the dimensions in user_order_lifetime.view.

#### Run Explore queries to test a new dimension

- Under Users > Dimensions, click Average Sales, and then click Run.

An error message is displayed along with the SQL query that the Explore sent to the underlying database. The error message identifies the problem at line 13: Query execution failed: - Syntax error: Expected end of input but got identifier "order_items" at [13:1].

- In the Data pane, open the SQL tab to more easily review the failed query, and review line 13:

order_items.user_id =user_order_lifetime.user_id

- Leave this browser tab for the Order Items Explore open, and open a new Looker window in a new tab.

- Navigate to Explore > Events.

- Under Users > Dimensions, click Average Sales, and then click Run.

- In the Data pane, open the SQL tab to see the successful query.

- Review line 13 in this query.

Unlike the query in the Order Items Explore, the query syntax at line 13 specifies a join between events and user_order_lifetime:

LEFT JOIN user_order_lifetime ON events.user_id =
      user_order_lifetime.user_id

#### Identify and correct invalid parameters in a model

- Return to the browser tab for the Looker IDE, and open training_ecommerce.model.

- Review the join for user_order_lifetime in both the order_items Explore and the events Explore.

- Review the documentation on parameters.

- In the explore: order_items definition, update the join for user_order_lifetime:

In [ ]:
join: user_order_lifetime {
  type: left_outer
  sql_on: ${order_items.user_id} = ${user_order_lifetime.user_id};;
  relationship: many_to_one
}

- Click Save Changes, and then click Validate LookML. There are no LookML errors.

- Return to the browser tab for the Order Items Explore, and refresh the page.

Now that you have correctly defined the join for user_order_lifetime within the order_items explore, the query runs successfully.

#### Save an Explore query as a Look

- Click once on the column for Average Sales to sort in descending order.

- Expand the query by clicking on additional dimensions: ID, State, Country, and Age.

- For Row Limit, enter: 10.

- Click Run.

- Expand the Visualization pane, and select the Table visualization.

- Click Settings (settings-gear-icon.png).

- Click Save > As a Look.

- Name the Look Top 10 Users With Highest Average Sales.

- Click Save & View Look. Your visualization should resemble the following:

- Close the other tab for the Explore, and leave this browser tab open as you begin the next task.

### Use the Content Validator to test and update content after changes to LookML objects

#### Modify the name of existing dimensions

- Open users.view, find the dimension called average_sales, and modify the name of the dimension to be more specific for business users:

In [ ]:
dimension: average_order_price  {
  type: number
  sql: ${user_order_lifetime.lifetime_sales} / ${user_order_lifetime.lifetime_orders} ;;
  value_format_name: usd
}

- Click Save Changes, and then click Validate LookML. There are no LookML errors.

- Leave this browser tab open for the IDE, return to the browser tab for the Look, and refresh the page.

Notice that there is now a warning: 'users.average_sales' no longer exists on Order Items, or you do not have access to it, and it will be ignored.

- Open a new Looker window in a new tab.

- Navigate to Develop > Content Validator.

- Click Validate.

The Error tab is active, and there is an error for “Unknown field "users.average_sales" for the Look called Top 10 Users With Highest Average Sales, which you created in the previous task.

- Click Find & Replace in All Content.

- For Type, select Field.

- For Field Name, type users.average_sales.

- For Replacement Field Name, type users.average_order_price.

- Click Replace Field Name.

- Click OK.

- Click Validate. The Error tab is now empty because the name of the dimension has been updated in all the content that referenced it (in this case, the Look named Top 10 Users With Highest Average Sales).

- Return to the browser tab for the Look, and refresh the page. The Look has been updated and is rendering the visualization successfully, so you can now push your LookML changes to production.

- Return to the browser tab for the IDE.

- Click Validate LookML. There should be no LookML errors.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes and Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

***

**<center><font size = "6">Employing Best Practices for Improving the Usability of LookML Projects<center>**
***

### Create fields that leverage existing fields and descriptive naming conventions

#### Create a yesno dimension from an existing dimension

- Click the toggle button to enter Development mode.

- From the Develop menu, select the qwiklabs_ecommerce project.

- Open order_items.view.

- Review the dimension named status.

In this case, ${TABLE}.status references the status column within the cloud-training-demos.looker_ecomm.order_items table. You can leverage this existing dimension to create a new yesno dimension that identifies whether an order has a canceled status.

Both order_is_canceled and is_order_canceled adhere to the best practice of descriptive names because they clearly identify the canceled object as an order. The choice between the two options is a naming convention decision that could be decided within your team.

- Under the status dimension, add the following code for a new yesno dimension:

In [ ]:
dimension: order_is_canceled {
	type: yesno
	sql: ${status} = "" ;;
}

In this case, ${status} references the status dimension within the order_items view.

Running a query on the order_items table directly in SQL Runner allows you to see the raw data values in the status column.

- Leave the browser tab for the IDE open, and open a new Looker window in a new browser tab.

- Navigate to Develop > SQL Runner.

- Click Settings (settings-gear-icon.png), and then click Search public projects.

- For Project, type cloud-training-demos, and then press ENTER.

- For Dataset, select looker_ecomm. A list of the available tables in this BigQuery dataset is displayed.

- To select the distinct values in the status column, add the following query to the SQL Query window, and then click Run:

In [ ]:
SELECT distinct(status)
FROM cloud-training-demos.looker_ecomm.order_items
ORDER BY status

Notice that the values include Cancelled, which uses the British English spelling with two letter Ls, instead of Canceled in the American English spelling.

- Close the browser tab for SQL Runner, and return to the browser tab for the IDE.

- Complete the LookML code for the new yesno dimension:

In [ ]:
dimension: order_is_canceled {
	type: yesno
	sql: ${status} = "Cancelled" ;;
}

- Click Save Changes. Leave this browser tab open for the Looker IDE.

- Open a new Looker window in a browser tab.

- Navigate to Explore > Order Items.

- In the Data pane, click on the SQL tab.

- Under Order Items > Dimensions, select:

- Order ID
- Order Is Canceled (Yes/No)

Before running the query, notice that the CASE statement returns the Yes or No result, depending on whether the value of order_item.status is equal to Cancelled:

In [ ]:
CASE WHEN order_items.status = "Cancelled"  THEN 'Yes' ELSE 'No' END

- Click Run.

- Open the Results tab to see the results.

- Close the browser tab for the Explore, and return to the browser tab for the IDE.

#### Create new measures based on an existing dimensions and measures


- Open order_items.view.

- Review the measure called total_revenue_from_completed_orders.

- Following this measure, add the following code to create two new measures:

In [ ]:
measure: total_revenue_from_canceled_orders {
	type: sum
	sql: ${sale_price} ;;
	filters: [order_is_canceled: "Yes"]
	value_format_name: usd
}
measure: percent_revenue_canceled_orders {
	type: number
	value_format_name: percent_2
	sql: 1.0*${total_revenue_from_canceled_orders}
	/NULLIF(${total_revenue}, 0) ;;
}

- Click Save Changes. Leave this browser tab open for the Looker IDE.

- Open a new Looker window in a browser tab.

- Navigate to Explore > Order Items.

- In the Data pane, click on the SQL tab.

- Under Order Items > Measures, select:

- Total Revenue From Canceled Orders

- Total Revenue

- Percent Revenue Canceled Orders

Before running the query, notice the CASE statement now used in conjunction with SUM to calculate the total of order_items.sale_price when the order_items.status is equal to Cancelled, and then divided by the total of all values in order_items.sale_price:

In [ ]:
SUM(CASE WHEN order_items.status = "Cancelled"  THEN
order_items.sale_price  ELSE NULL END), 0) / NULLIF(COALESCE(SUM(order_items.sale_price ), 0), 0)

- Click Run.

- Open the Results tab to see the results.

- Close the browser tab for the Explore query, and return to the browser tab with the Looker IDE.

- Click Project Health (project-health.png).

- In the Project Health > LookML validation section, click Validate LookML. No LookML errors should be found.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes and Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

### Provide context to fields and Explores with labels and descriptions

#### Add labels and descriptions to dimensions

- Open a new Looker window in a new browser tab.

- Navigate to Explore > Order Items.

- Expand Order Items, and hold the pointer over the dimension called Order Is Canceled (Yes/No) to see additional options.

- Click Info (info icon) to see the details of this dimension.

- Leave the browser tab for the Explore open, and return to the browser tab for the Looker IDE.

- Open order_items.view.

- Add a description for the order_is_canceled dimension you created earlier:

In [ ]:
dimension: order_is_canceled {
    description: "A value equal to Yes means that the order
has a canceled status. A value equal to No means that the
order does not have a canceled status."
    type: yesno
    sql: ${status} = "Cancelled" ;;
  }

- Click Save Changes, and then click Validate LookML.

- Return to the browser tab for the Order Items Explore, and refresh the page.

- Expand Order Items, and hold the pointer over the dimension called Order Is Canceled (Yes/No) to see additional options.

- Click Info (info icon) to see the details of this dimension.

- Leave the browser tab for the Explore open, and return to the browser tab for the Looker IDE.

- Review the LookML code for order_is_canceled dimension again.

#### Add labels and descriptions to measures


- In order_items.view, add descriptions and labels to the measures called total_revenue_from_canceled_orders and percent_revenue_canceled_orders:

In [ ]:
measure: total_revenue_from_canceled_orders {
    label: "Total Revenue Lost From Canceled Orders"
    description: "Sum of sale price for orders with canceled status."
    type: sum
    sql: ${sale_price} ;;
    filters: [order_is_canceled: "Yes"]
    value_format_name: usd
}
measure: percent_revenue_canceled_orders {
    label: "% Revenue Lost From Canceled Orders"
    description: "Total revenue lost from canceled orders as
    a percentage of the total revenue from all orders."
    type: number
    value_format_name: percent_2
    sql: 1.0*${total_revenue_from_canceled_orders}
      /NULLIF(${total_revenue}, 0) ;;
}

- Click Save Changes, and then click Validate LookML.

- Return to the browser tab for the Order Items Explore, and refresh the page.

- Expand Order Items, and notice the revised labels for the two measures: % Revenue Lost From Canceled Orders and Total Revenue Lost From Canceled Orders.

- Click Info (info icon) for each measure to see the details.

- Under Order Items > Dimensions, select: Order Is Canceled (Yes/No).

- Under Order Items > Measures, select:

- Total Revenue Lost From Canceled Orders

- Total Revenue

- % of Revenue Lost from Canceled Orders

- In the Data pane, hover over the column names to see the same description provided for the Info button for Order Is Canceled, Total Revenue Lost From Canceled Orders, and % of Revenue Lost from Canceled Orders.

- Click Run.

#### Add a label and description to an Explore

- Close the browser tab for the Explore, and return to the browser tab for the Looker IDE.

- Open training_ecommerce.model.

- Add a label and description to the existing Order Items Explore, before the join for users:

In [ ]:
explore: order_items {
  label: "Orders and Users"
  description: "Use this Explore to review details for orders and users,
  including information on inventory, products, and distribution centers."
  join: users {
    type: left_outer
    sql_on: ${order_items.user_id} = ${users.id} ;;
    relationship: many_to_one
}

- Click Save Changes, and then click Validate LookML.

- Leave the browser tab for the IDE open, and open a new Looker window in a new browser tab.

- Click Explore to see the menu list of Explores.

- Hold the pointer over Info (info icon) to see the details of the Explore.

- Close the browser tab for the Explore, and leave the browser tab for the IDE open.

### Limit fields to only those needed in a specific Explore

#### Hide a field from all Explores in a LookML model

- Open a new Looker window in a new browser tab.

- Navigate to Explore > Events.

- Expand Users.

- Open another new Looker window in a new browser tab.

- Navigate to Explore > Orders and Users.

- Expand Users, and compare the two Explores.

- Leave the browser tabs open for both Explores, and return to the Looker IDE.

- Open users.view.

- Add the hidden parameter to the latitude and longitude dimensions:

In [ ]:
dimension: latitude {
  hidden: yes
  type: number
  sql: ${TABLE}.latitude ;;
}
dimension: longitude {
  hidden:  yes
  type: number
  sql: ${TABLE}.longitude ;;
}

- Click Save Changes, and then click Validate LookML.

- Return to the browser tab for the Orders and Users Explore, and refresh the page.

- Expand Users, and review the available dimensions.

- Return to the browser tab for the Events Explore, and refresh the page.

- Expand Users, and review the available dimensions.

#### Selectively hide fields from specific Explores

- Leave the browser tabs open for the Explore, and return to the browser tab for the IDE.

- Open training_ecommerce.model.

- Add the fields parameter to the existing Events Explore, before the join for event_session_facts:

In [ ]:
explore: events {
  fields: [ALL_FIELDS*, -users.city, -users.email, -users.first_name,
    -users.gender, -users.last_name, -users.state]
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }

- Click Save Changes, and then click Validate LookML.

- Return to the browser tab for the Orders and Users Explore, and refresh the page.

- Expand Users, and review the available dimensions.

- Return to the browser tab for the Events Explore, and refresh the page.

- Expand Users, and review the available dimensions.

- Under Users > Dimensions, select Country.

- Under Users > Measures, select Count.

- Under Events > Dimensions, select Event Type.

- Click Run.

- Close both of the browser tabs open for the Explore, and leave the browser tab for the IDE open.

### Group similar fields or Explores into useful categories

#### Group similar fields in a view

- Open users.view and review the available dimensions.

- Add the group_label parameter to the city, country, state, and zip dimensions:

In [ ]:
dimension: city {
  group_label: "Location"
  type: string
  sql: ${TABLE}.city ;;
}
dimension: country {
  group_label: "Location"
  type: string
  map_layer_name: countries
  sql: ${TABLE}.country ;;
}
dimension: state {
  group_label: "Location"
  type: string
  sql: ${TABLE}.state ;;
  map_layer_name: us_states
}
dimension: zip {
  group_label: "Location"
  type: zipcode
  sql: ${TABLE}.zip ;;
  }

- Click Save Changes, and then click Validate LookML.

- Leave the browser tab open for the IDE, and open a new Looker window in a new browser tab.

- Navigate to Explore > Orders and Users.

- Expand Users and review the available dimensions.

- Under Users > Dimension > Location, select:

- City

- Country

- State

- Zip

- Click Run to see the results.

- Close the browser tab for the Orders and Users Explore, and return to the browser tab for the IDE.

#### Create groups of Explores under different headings


- Open training_ecommerce.model.

- Before the label parameter, add a group_label called “E-commerce - Inventory Team” to the Orders and Users Explore:

In [ ]:
explore: order_items {
  group_label: "E-commerce - Inventory Team"
  label: "Orders and Users"

- Before the fields parameter, add a group_label called “E-commerce - Marketing Team” to the Events Explore:

In [ ]:
explore: events {
  group_label: "E-commerce - Marketing Team"
  fields: [ALL_FIELDS*, -users.city, -users.email, -users.first_name,
    -users.gender, -users.last_name, -users.state]

- Click Save Changes, and then click Validate LookML.

- Open a new Looker window in a new browser tab.

- Expand the Explore menu and review the options.

##### training_ecommerce.model

In [ ]:
connection: "bigquery_public_data_looker"

# include all the views
include: "/views/*.view"
include: "/z_tests/*.lkml"
include: "/**/*.dashboard"

datagroup: training_ecommerce_default_datagroup {
  # sql_trigger: SELECT MAX(id) FROM etl_log;;
  max_cache_age: "1 hour"
}

persist_with: training_ecommerce_default_datagroup

label: "E-Commerce Training"

explore: order_items {
  group_label: "E-commerce - Inventory Team"
  label: "Orders and Users"
  description: "Use this Explore to review details for orders and users,
  including information on inventory, products, and distribution centers."
  join: users {
    type: left_outer
    sql_on: ${order_items.user_id} = ${users.id} ;;
    relationship: many_to_one
  }

  join: inventory_items {
    type: left_outer
    sql_on: ${order_items.inventory_item_id} = ${inventory_items.id} ;;
    relationship: many_to_one
  }

  join: products {
    type: left_outer
    sql_on: ${inventory_items.product_id} = ${products.id} ;;
    relationship: many_to_one
  }

  join: distribution_centers {
    type: left_outer
    sql_on: ${products.distribution_center_id} = ${distribution_centers.id} ;;
    relationship: many_to_one
  }
}

explore: events {
  group_label: "E-commerce - Marketing Team"
  fields: [ALL_FIELDS*, -users.city, -users.email, -users.first_name,
    -users.gender, -users.last_name, -users.state]
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }
}


##### order_items.views

In [ ]:
view: order_items {
  sql_table_name: `cloud-training-demos.looker_ecomm.order_items`
    ;;
  drill_fields: [order_item_id]

  dimension: order_item_id {
    primary_key: yes
    type: number
    sql: ${TABLE}.id ;;
  }

  dimension_group: created {
    type: time
    timeframes: [
      raw,
      time,
      date,
      week,
      month,
      quarter,
      year
    ]
    sql: ${TABLE}.created_at ;;
  }

  dimension_group: delivered {
    type: time
    timeframes: [
      raw,
      date,
      week,
      month,
      quarter,
      year
    ]
    convert_tz: no
    datatype: date
    sql: ${TABLE}.delivered_at ;;
  }

  dimension: inventory_item_id {
    type: number
    # hidden: yes
    sql: ${TABLE}.inventory_item_id ;;
  }

  dimension: order_id {
    type: number
    sql: ${TABLE}.order_id ;;
  }

  dimension_group: returned {
    type: time
    timeframes: [
      raw,
      time,
      date,
      week,
      month,
      quarter,
      year
    ]
    sql: ${TABLE}.returned_at ;;
  }

  dimension: sale_price {
    type: number
    sql: ${TABLE}.sale_price ;;
  }

  dimension_group: shipped {
    type: time
    timeframes: [
      raw,
      date,
      week,
      month,
      quarter,
      year
    ]
    convert_tz: no
    datatype: date
    sql: ${TABLE}.shipped_at ;;
  }

  dimension: order_is_canceled {
    description: "A value equal to Yes means that the order
    has a canceled status. A value equal to No means that the
    order does not have a canceled status."
    type: yesno
    sql: ${TABLE}.status  = "Cancelled" ;;
  }

  dimension: user_id {
    type: number
    # hidden: yes
    sql: ${TABLE}.user_id ;;
  }


  measure: average_sale_price {
    type: average
    sql: ${sale_price} ;;
    drill_fields: [detail*]
    value_format_name: usd_0
  }

  measure: order_item_count {
    type: count
    drill_fields: [detail*]
  }

  measure: order_count {
    type: count_distinct
    sql: ${order_id} ;;
  }

  measure: total_revenue {
    type: sum
    sql: ${sale_price} ;;
    value_format_name: usd
  }

  measure: total_revenue_from_completed_orders {
    type: sum
    sql: ${order_items.sale_price} and ${TABLE}.status  = "Complete";;
    # filters: [${TABLE}.status : "Complete"]
    value_format_name: usd
  }

  measure: total_revenue_from_canceled_orders {
    label: "Total Revenue Lost From Canceled Orders"
    description: "Sum of sale price for orders with canceled status."
    type: sum
    sql: ${sale_price} ;;
    filters: [order_is_canceled: "Yes"]
    value_format_name: usd
  }
  measure: percent_revenue_canceled_orders {
    label: "% Revenue Lost From Canceled Orders"
    description: "Total revenue lost from canceled orders as
    a percentage of the total revenue from all orders."
    type: number
    value_format_name: percent_2
    sql: 1.0*${total_revenue_from_canceled_orders}
      /NULLIF(${total_revenue}, 0) ;;
  }


  # ----- Sets of fields for drilling ------
  set: detail {
    fields: [
      order_item_id,
      users.last_name,
      users.id,
      users.first_name,
      inventory_items.id,
      inventory_items.product_name
    ]
  }
}


##### users.view

In [ ]:
view: users {
  sql_table_name: `cloud-training-demos.looker_ecomm.users`
    ;;
  drill_fields: [id]

  dimension: id {
    primary_key: yes
    type: number
    sql: ${TABLE}.id ;;
  }

  dimension: age {
    type: number
    sql: ${TABLE}.age ;;
  }

  dimension: city {
    group_label: "Location"
    type: string
    sql: ${TABLE}.city ;;
  }

  dimension: country {
    group_label: "Location"
    type: string
    map_layer_name: countries
    sql: ${TABLE}.country ;;
  }

  dimension_group: created {
    type: time
    timeframes: [
      raw,
      time,
      date,
      week,
      month,
      quarter,
      year
    ]
    sql: ${TABLE}.created_at ;;
  }

  dimension: email {
    type: string
    sql: ${TABLE}.email ;;
  }

  dimension: first_name {
    type: string
    sql: ${TABLE}.first_name ;;
  }

  dimension: gender {
    type: string
    sql: ${TABLE}.gender ;;
  }

  dimension: last_name {
    type: string
    sql: ${TABLE}.last_name ;;
  }

  dimension: latitude {
    hidden: yes
    type: number
    sql: ${TABLE}.latitude ;;
  }
  dimension: longitude {
    hidden:  yes
    type: number
    sql: ${TABLE}.longitude ;;
  }

  dimension: state {
    group_label: "Location"
    type: string
    sql: ${TABLE}.state ;;
    map_layer_name: us_states
  }

  dimension: traffic_source {
    type: string
    sql: ${TABLE}.traffic_source ;;
  }

  dimension: zip {
    group_label: "Location"
    type: zipcode
    sql: ${TABLE}.zip ;;
  }




  measure: count {
    type: count
    drill_fields: [id, last_name, first_name, events.count, order_items.count]
  }
}


***

**<center><font size = "6">Caching and Datagroups with LookML<center>**
***

### Apply a datagroup to an Explore

#### Open the model

- Click the toggle button to enter Development mode.

- On the Develop tab, select the qwiklabs-ecommerce LookML project.

- Open the training_ecommerce.model file.

Notice that this model file has a default datagroup with a max_cache_age of 1 hour. Whenever you create a new LookML project by having Looker generate the model from the database schema, Looker automatically creates a default datagroup with the name of the model, in this case training_ecommerce, followed by _default_datagroup.

#### Delete the default datagroup and replace it

Because this default datagroup is currently defined at the model level, it is applied to all Explores defined in the model. You want to apply the datagroup to the Explore, so you must remove the default one and update it accordingly. To complete the definition of the new datagroup, you need to provide values for the two parameters: sql_trigger and max_cache_age.

- Delete the default datagroup and the persist_with definition (Lines 8-13).

- To create a new datagroup for a specific Explore such as Order Items, enter the following code:

In [ ]:
datagroup: order_items_datagroup {}

- For the sql_trigger, to select the maximum ID for order_item_id, enter the following code:

In [ ]:
sql_trigger: SELECT MAX(order_item_id) from order_items ;;

- Set the max_cache_age so that caching will continue to refresh every hour even if there is an issue with data updates. Enter the following code:

In [ ]:
max_cache_age: "1 hour"

#### Apply the datagroup

Note that configuring a datagroup by itself doesn’t do anything; it is a two-step process. After defining the datagroup, you need to apply the datagroup to an object by using a parameter called persist_with.

- To apply the datagroup to the definition for the Order Items Explore, directly under the explore: order_items line, enter the following code:

In [ ]:
persist_with: order_items_datagroup

- Click Save Changes, and then click Validate LookML.

- Navigate back to the training_ecommerce.model file.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes and Push.

- commit-changes.png

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

In [ ]:
connection: "bigquery_public_data_looker"

# include all the views
include: "/views/*.view"
include: "/z_tests/*.lkml"
include: "/**/*.dashboard"

datagroup: order_items_datagroup {
  sql_trigger: SELECT MAX(order_item_id) from order_items ;;
  max_cache_age: "1 hour"
}
label: "E-Commerce Training"

explore: order_items {
  persist_with: order_items_datagroup
  join: users {
    type: left_outer
    sql_on: ${order_items.user_id} = ${users.id} ;;
    relationship: many_to_one
  }

  join: inventory_items {
    type: left_outer
    sql_on: ${order_items.inventory_item_id} = ${inventory_items.id} ;;
    relationship: many_to_one
  }

  join: products {
    type: left_outer
    sql_on: ${inventory_items.product_id} = ${products.id} ;;
    relationship: many_to_one
  }

  join: distribution_centers {
    type: left_outer
    sql_on: ${products.distribution_center_id} = ${distribution_centers.id} ;;
    relationship: many_to_one
  }
}

explore: events {
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }
}


***

**<center><font size = "6">Optimizing Performance of LookML Queries<center>**
***

### Create an incremental persistent derived table that will auto-update without rebuilding the entire table

#### Use an Explore to create a native derived table

- Click the toggle button to enter Development mode.

- Navigate to Explore > Order Items.

- Under Order Items > Dimensions, select the following:

- Order ID

- Sale Price

- Created Date > Date

- Created Date > Week

- Created Date > Month

- Under Users > Dimensions, select State.

- Click Run.

- Click Settings (Settings gear icon).

- Select Get LookML.

- On the tab for Derived Table, copy the LookML code to a text editor. You'll use this code to create a new view for the native derived table.

In [ ]:
# If necessary, uncomment the line below to include explore_source.
# include: "training_ecommerce.model.lkml"

view: add_a_unique_name_1666920690 {
  derived_table: {
    explore_source: order_items {
      column: order_id {}
      column: sale_price {}
      column: created_date {}
      column: created_week {}
      column: created_month {}
      column: state { field: users.state }
    }
  }
  dimension: order_id {
    description: ""
    type: number
  }
  dimension: sale_price {
    description: ""
    type: number
  }
  dimension: created_date {
    description: ""
    type: date
  }
  dimension: created_week {
    description: ""
    type: date_week
  }
  dimension: created_month {
    description: ""
    type: date_month
  }
  dimension: state {
    description: ""
  }
}


#### Create a view file for a derived table

- Open a new Looker window in a new browser tab.

- On the Develop menu, click qwiklabs_ecommerce.

- Click the plus icon (+) next to File Browser, and select Create View.

- Name the new file incremental_pdt, and click Create.

- In the File Browser, click on incremental_pdt.view and drag it under the views folder.

- Replace the default LookML code in incremental_pdt.view with the code that you copied previously for the native derived table.

- Update line 4 with the correct view name (incremental_pdt).

- Update the order_id dimension to define it as the primary_key for the view:

In [ ]:
dimension: order_id {
    primary_key:  yes
    type: number
}

This is because each record represents an order with unique order_id.

- Find the last dimension, and add two new measures before the final closing curly bracket (}) in the file:

In [ ]:
measure: average_sale_price {
    type: average
    sql: ${sale_price} ;;
    value_format_name: usd_0
  }
  measure: total_revenue {
    type: sum
    sql: ${sale_price} ;;
    value_format_name: usd
  }

- Click Save Changes. You file should resemble the following:

#### Add persistence and incremental updates to a derived table

- Open training_ecommerce.model.

- Find the default datagroup named training_ecommerce_default_datagroup, and add a new line (line 13).

- Define a new datagroup to persist objects with daily refresh (max time of 24 hours):

In [ ]:
datagroup: daily_datagroup {
  sql_trigger: SELECT FORMAT_TIMESTAMP('%F',
CURRENT_TIMESTAMP(), 'America/Los_Angeles') ;;
  max_cache_age: "24 hours"
}

The sql_trigger checks the current date and triggers a refresh when the date changes, and max_cache_age ensures that the table will rebuild after 24 hours, even if the sql_trigger fails to run successfully.

- At the end of training_ecommerce.model (around line 67), define a new Explore that contains only the incremental_pdt view so that you can test it in subsequent steps:

In [ ]:
explore: incremental_pdt {}

- Click Save Changes.

- Open incremental_pdt.view, and add persistence by including the daily datagroup in the derived table definition at line 6:

In [ ]:
datagroup_trigger: daily_datagroup

- Add incremental updates by including the following parameters in the derived table definition at lines 7 and 8:

In [ ]:
increment_key: "created_date"
increment_offset: 3

- Click Save Changes. Your file should resemble the following:

The persistent derived table will now be persisted and will rebuild once a day, going back 3 days to capture any orders that may have arrived late.

- Close the browser tab for the original Explore query, but leave the tab open for the Looker IDE.

#### Test Explore queries on a persistent incremental derived table

- Open a new Looker window in a browser tab.

- Navigate to Explore > Incremental Pdt.

- In the Data pane, open the SQL tab.

- Under Incremental Pdt > Dimensions, select Created Date.

- Under Incremental Pdt > Measures, select Average Sale Price and Total Revenue.

Before running the query, notice that there are two queries in the SQL window (which may take a few seconds to load). The first query generates the PDT named incremental_pdt, and the second query retrieves the results from the newly created PDT.

- Click Run.

- Open the Results tab to see the results.

- Under Incremental Pdt > Dimensions:

- Clear Created Date.

- Select Created Month.

- In the Data pane, open the SQL tab.

Notice that the query will use the same PDT to retrieve the results, which makes sense because you requested a time frame that is already defined (and cached) in the PDT. However, notice that you cannot select and run a query on a different time frame that is not already included in the PDT, such as Quarter or Year.

- Click Run.

- Open the Results tab to see the results.

#### Challenge

- Run a new query using only the State dimension and the Average Sale Price and Total Revenue measures. Answer the following question.

- Close the browser tab for the Explore query, and return to the browser tab with the Looker IDE.

- Click Validate LookML. There should be no LookML errors.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

Remain in the browser tab for the Looker IDE as you begin the next task.

### Create an incremental aggregate table to summarize order data across multiple time periods

#### Create an aggregate table within a refinement of an existing Explore

- From the Looker IDE page, open training_ecommerce.model.

- At the end of the file (around line 69), add the following code to create a refinement of the order_items Explore:

In [ ]:
explore: +order_items {
    label: "Order Items - Aggregate Sales"
}

- Expand the LookML code for the refinement to include an aggregate table that summarizes order data by time frame or state:

In [ ]:
explore: +order_items {
    label: "Order Items - Aggregate Sales"
    aggregate_table: aggregate_sales {
        query: {
            dimensions: [order_items.created_date, users.state]
            measures: [order_items.average_sale_price,
order_items.total_revenue]
        }
        materialization: {
            datagroup_trigger: daily_datagroup
     	 increment_key: "created_date"
      	increment_offset: 3
        }
    }
}

- Click Save Changes.

#### Test Explore queries on a persistent incremental aggregate table

- Open a new Looker window in a browser tab.

- Navigate to Explore > Order Items - Aggregate Sales.

- In the Data pane, open the SQL tab.

- Under Order Items > Dimensions, select Created Date > Date.

- Under Order Items > Measures, select Average Sale Price and Total Revenue.

- Click Run.

- Open the Results tab to see the results.

- Under Order Items > Dimensions > Created Date:

- Clear Date.

- Select Quarter.

- In the Data pane, open the SQL tab.

- Click Run.

- Open the Results tab to see the results.

#### Challenge

- Run a new query using only the State dimension (under Users) and the Average Sale Price and Total Revenue measures. Answer the following question.

- Run a new query using only the Country dimension (under Users) and the Average Sale Price and Total Revenue measures. Answer the following question.

- Close the browser tab for the Explore query, and return to the browser tab with the Looker IDE.

- Click Validate LookML. There should be no LookML errors.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

### Join views in a performant manner to optimize Explore queries

#### Identify the most appropriate field to use as the primary key of a view

- Open the users.view file. Answer the following question.

- Open the order_items.view file. Answer the following question.

- Open a new Looker window in a new browser tab.

- Navigate to Develop > SQL Runner.

- Click Settings (Settings gear icon) next to Connection, and select Search public projects.

- The box for Project will now be empty. Type cloud-training-demos, and press ENTER.

- For Dataset, select looker_ecomm. A list of the available tables in this BigQuery dataset is displayed.

- To check whether the user_id column would make an appropriate primary key, add the following query to the SQL Query window, and click Run:

In [ ]:
SELECT count(*), count(distinct user_id)
FROM cloud-training-demos.looker_ecomm.order_items

- Repeat the query for the order_id, inventory_item_id, and id columns.

- Close the browser tab for SQL Runner, and return to the browser tab with the Looker IDE.

#### Join the minimal amount of views to define new Explores

- Open training_ecommerce.model.

- Review the existing order_items Explore.

- At the end of the file (around line 85) add the following code to define a new Explore with order_items as the base view and only the users view joined:

In [ ]:
explore: aggregated_orders {
  from: order_items
  label: "Aggregated Sales"
  join: users {
    type: left_outer
    sql_on: ${aggregated_orders.user_id} = ${users.id} ;;
    relationship: one_to_many
  }
  aggregate_table: aggregate_sales {
        query: {
            dimensions: [aggregated_orders.created_date,
users.state]
            measures: [aggregated_orders.average_sale_price,
aggregated_orders.total_revenue]
        }
        materialization: {
            datagroup_trigger: daily_datagroup
     	 	increment_key: "created_date"
     	 	increment_offset: 3
        }
    }
  }

- Click Save Changes. Your file should resemble the following:

#### Define performant join relationships for efficient Explore queries

- Open a new Looker window in a new browser tab.

- Navigate to Explore > Aggregated Sales.

- In the Data pane, open the SQL tab.

- Under Aggregated Orders > Dimensions, select Created Date > Date.

- Under Aggregated Orders > Measures, select:

- Average Sale Price

- Total Revenue

Before running the query, notice that the aggregate table is not being used due to an issue with a join fanout:


-- Did not use aggregated_orders::aggregate_sales; field aggregated_orders.average_sale_price was DISTINCT in the table due to a join fanout, but there was no fanout in the query

An unintended fanout can occur when the relationship between two tables is not identified correctly for a join. In this case, the base view of the Explore is order_items, which can contain many orders for one user. However, the users view contains only one record for each user.

Therefore, this join should actually be defined as many_to_one, or many orders to one user, instead of one order to many users. (Learn more about the problem of fanouts in the Looker Help Center.)

- Click Run.

- Open the Results tab. The results are returned, but Looker did not use the efficient aggregate table to retrieve the results.

- Leave this browser tab for the Explore open, and return to the browser tab with the Looker IDE.

- Update the relationship parameter to many_to_one (line 91) in the aggregated_orders explore:

In [ ]:
relationship: many_to_one

- Click Save Changes. Your file should resemble the following:

- Return to the browser tab for the Explore query, and refresh the page.

- In the Data pane, open the SQL tab.

- Open the Results tab to see the results.

- Close the browser tab for the Explore query, and return to the browser tab with the Looker IDE.

- Click Validate LookML. There should be no LookML errors.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

#### Monitor the builds of persistent derived tables in a Looker instance


#### Review the status of PDTs in production

- Open a new Looker window in a new browser tab.

- Navigate to Admin > Persistent Derived Tables. No PDTs are listed in the Development tab because all of your PDTs have been pushed to production.

- Open the Production tab to see the PDTs you created in Tasks 1-3.

#### Modify and review PDTs in development

- Return to the browser tab with the Looker IDE.

- Open training_ecommerce.model.

- Add a new dimension for users.country to the aggregated_orders Explore (around line 96):

In [ ]:
dimensions: [aggregated_orders.created_date, users.state, users.country]

- Click Save Changes.

- Return to the Persistent Derived Tables page, and refresh the page.

- Open the Development tab.

- Leave this Persistent Derived Tables page open, and open a new Looker window in a new browser tab.

- Navigate to Explore > Aggregate Sales.

- In the Data pane, open the SQL tab.

- Under Users > Dimensions, select Country.

- Under Aggregated Orders > Measures, select:

- Average Sale Price

- Total Revenue

- Click Run.

- Open the Results tab to see the results.

- Close the browser tab for the Explore query, return to the browser tab with the Persistent Derived Tables page, and refresh the page.

- Leave the browser tab for the Persistent Derived Tables page open, and return to the browser tab with the Looker IDE.

- Click Validate LookML. There are no LookML errors.

#### Commit changes and deploy to production

- Click Validate LookML and then click Commit Changes & Push.

- Add a commit message and click Commit.

- Lastly, click Deploy to Production.

##### incremental_pdt.views

In [ ]:
# If necessary, uncomment the line below to include explore_source.
# include: "training_ecommerce.model.lkml"

view: incremental_pdt {
  derived_table: {
    datagroup_trigger: daily_datagroup
    increment_key: "created_date"
    increment_offset: 3
    explore_source: order_items {
      column: order_id {}
      column: sale_price {}
      column: created_date {}
      column: created_week {}
      column: created_month {}
      column: state { field: users.state }
    }
  }
  dimension: order_id {
    primary_key:  yes
    type: number
  }
  dimension: sale_price {
    description: ""
    type: number
  }
  dimension: created_date {
    description: ""
    type: date
  }
  dimension: created_week {
    description: ""
    type: date_week
  }
  dimension: created_month {
    description: ""
    type: date_month
  }
  dimension: state {
    description: ""
  }
  measure: average_sale_price {
    type: average
    sql: ${sale_price} ;;
    value_format_name: usd_0
  }
  measure: total_revenue {
    type: sum
    sql: ${sale_price} ;;
    value_format_name: usd
  }
}


##### training_ecommerce.model

In [ ]:
connection: "bigquery_public_data_looker"

# include all the views
include: "/views/*.view"
include: "/z_tests/*.lkml"
include: "/**/*.dashboard"

datagroup: training_ecommerce_default_datagroup {
  # sql_trigger: SELECT MAX(id) FROM etl_log;;
  max_cache_age: "1 hour"
}

datagroup: daily_datagroup {
  sql_trigger: SELECT FORMAT_TIMESTAMP('%F',
    CURRENT_TIMESTAMP(), 'America/Los_Angeles') ;;
  max_cache_age: "24 hours"
}
persist_with: training_ecommerce_default_datagroup

label: "E-Commerce Training"

explore: order_items {
  join: users {
    type: left_outer
    sql_on: ${order_items.user_id} = ${users.id} ;;
    relationship: many_to_one
  }

  join: inventory_items {
    type: left_outer
    sql_on: ${order_items.inventory_item_id} = ${inventory_items.id} ;;
    relationship: many_to_one
  }

  join: products {
    type: left_outer
    sql_on: ${inventory_items.product_id} = ${products.id} ;;
    relationship: many_to_one
  }

  join: distribution_centers {
    type: left_outer
    sql_on: ${products.distribution_center_id} = ${distribution_centers.id} ;;
    relationship: many_to_one
  }
}

explore: events {
  join: event_session_facts {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_facts.session_id} ;;
    relationship: many_to_one
  }
  join: event_session_funnel {
    type: left_outer
    sql_on: ${events.session_id} = ${event_session_funnel.session_id} ;;
    relationship: many_to_one
  }
  join: users {
    type: left_outer
    sql_on: ${events.user_id} = ${users.id} ;;
    relationship: many_to_one
  }
}


explore: incremental_pdt {}

explore: +order_items {
  label: "Order Items - Aggregate Sales"
  aggregate_table: aggregate_sales {
    query: {
      dimensions: [order_items.created_date, users.state]
      measures: [order_items.average_sale_price,
        order_items.total_revenue]
    }
    materialization: {
      datagroup_trigger: daily_datagroup
      increment_key: "created_date"
      increment_offset: 3
    }
  }
}

explore: aggregated_orders {
  from: order_items
  label: "Aggregated Sales"
  join: users {
    type: left_outer
    sql_on: ${aggregated_orders.user_id} = ${users.id} ;;
    relationship: many_to_one
  }
  aggregate_table: aggregate_sales {
    query: {
      dimensions: [aggregated_orders.created_date, users.state, users.country]
      measures: [aggregated_orders.average_sale_price,
        aggregated_orders.total_revenue]
    }
    materialization: {
      datagroup_trigger: daily_datagroup
      increment_key: "created_date"
      increment_offset: 3
    }
  }
}


***